In [145]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import kagglehub
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split





In [146]:
# importing datasets

CC_data = pd.read_csv("cc_institution_details.csv")

path = kagglehub.dataset_download("benroshan/factors-affecting-campus-placement")
print("Path to dataset files:", path)
Placement_data = pd.read_csv("Placement_Data_Full_Class.csv")

Path to dataset files: /Users/williamhaufrect/.cache/kagglehub/datasets/benroshan/factors-affecting-campus-placement/versions/1


In [147]:
CC_data.head()

,index,unitid,chronname,city,state,level,control,basic,hbcu,flagship,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


In [148]:
Placement_data.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


# Problems and questions raised from the datasets

Question for the CC institution dataset: How does private vs. public control affect graduation rates?

Question for the Placement dataset: How does adding a specialization change expected salary?

Problem brainstorm: The Placement dataset shows student percentiles, degree specialization, and offered salaries. It can be used to analyze how these factors affect starting salary. For the CC institution dataset, one interesting focus is graduation rate and the differences between public and private institutions.

# Step Two

Write a generic question that this dataset could address: What variables affect salary and job compensation?

Independent business metric: The independent business metric I will use is the mean of the salary offered to candidates.

In [149]:
CC_data.info()
Placement_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 63 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   index                                 3798 non-null   int64  
 1   unitid                                3798 non-null   int64  
 2   chronname                             3798 non-null   str    
 3   city                                  3798 non-null   str    
 4   state                                 3798 non-null   str    
 5   level                                 3798 non-null   str    
 6   control                               3798 non-null   str    
 7   basic                                 3798 non-null   str    
 8   hbcu                                  94 non-null     str    
 9   flagship                              50 non-null     str    
 10  long_x                                3798 non-null   float64
 11  lat_y                       

In [150]:
change_col = ["gender","ssc_b","hsc_b","hsc_s", "degree_t","workex","specialisation","status"]

Placement_data[change_col] = Placement_data[change_col].astype("category")

Placement_data[change_col].dtypes


gender            category
ssc_b             category
hsc_b             category
hsc_s             category
degree_t          category
workex            category
specialisation    category
status            category
dtype: object

In [151]:
change_col2 = ["level","control","basic","hbcu","flagship","state","city","site","similar","nicknames"]

CC_data[change_col2] = CC_data[change_col2].astype("category")

CC_data[change_col2].dtypes


level        category
control      category
basic        category
hbcu         category
flagship     category
state        category
city         category
site         category
similar      category
nicknames    category
dtype: object

In [152]:
state_region = {
    "ME":"Northeast","NH":"Northeast","VT":"Northeast","MA":"Northeast", "RI":"Northeast","CT":"Northeast",
    "NY":"Northeast","NJ":"Northeast","PA":"Northeast",

    "DE":"South","MD":"South","DC":"South","VA":"South","WV":"South", "NC":"South","SC":"South","GA":"South","FL":"South", 
    "KY":"South","TN":"South","AL":"South","MS":"South","AR":"South","LA":"South","OK":"South","TX":"South",

    "OH":"Midwest","IN":"Midwest","MI":"Midwest","WI":"Midwest", "IL":"Midwest","MN":"Midwest","IA":"Midwest","MO":
    "Midwest", "ND":"Midwest","SD":"Midwest","NE":"Midwest","KS":"Midwest",
    
    "MT":"West","WY":"West","CO":"West","NM":"West","AZ":"West","UT":"West","NV":"West","ID":"West",
    "WA":"West","OR":"West","CA":"West","AK":"West","HI":"West"
}

CC_data["region"] = CC_data["state"].map(state_region)
CC_data["region"] = CC_data["region"].astype("category")




In [153]:
category_list_cc = list(CC_data.select_dtypes('category'))

CC_data_hot_coded = pd.get_dummies(CC_data,
    columns= category_list_cc,
    drop_first=True)

In [154]:
category_list_place = list( Placement_data.select_dtypes('category'))

Placement_hot_coded = pd.get_dummies(
    Placement_data,
    columns= category_list_place,
    drop_first=True
)


In [155]:
num_cols = ["ssc_p", "hsc_p", "degree_p", "etest_p", "mba_p"]
Placement_hot_coded[num_cols] = MinMaxScaler().fit_transform(Placement_hot_coded[num_cols])

Placement_hot_coded[num_cols].describe()


,ssc_p,hsc_p,degree_p,etest_p,mba_p
count,215.000000,215.000000,215.000000,215.000000,215.000000
mean,0.544494,0.483248,0.399273,0.460428,0.414850
std,0.223195,0.179531,0.179482,0.276582,0.218643
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.406308,0.393740,0.268293,0.208333,0.252436
50%,0.538240,0.461285,0.390244,0.437500,0.404423
75%,0.717584,0.593081,0.536585,0.697917,0.563906
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [156]:
target = "grad_150_value"

num_cols_cc = (CC_data_hot_coded.select_dtypes(include=["int64","float64"]).columns.drop(target))
CC_data_hot_coded[num_cols_cc] = MinMaxScaler().fit_transform(CC_data_hot_coded[num_cols_cc])
CC_data_hot_coded[num_cols_cc].describe()


,index,unitid,long_x,lat_y,student_count,awards_per_value,awards_per_state_value,awards_per_natl_value,exp_award_value,exp_award_state_value,...,vsa_grad_elsewhere_after4_transfer,vsa_enroll_after4_transfer,vsa_enroll_elsewhere_after4_transfer,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,state_sector_ct,carnegie_ct,cohort_size
count,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,3798.000000,...,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,3798.000000,3798.000000,3467.000000
mean,0.500000,0.334750,0.741873,0.357081,0.026176,0.167288,0.346484,0.367119,0.012320,0.277221,...,0.459311,0.189670,0.341575,0.649052,0.456242,0.174505,0.277447,0.289429,0.448755,0.040504
std,0.288789,0.280447,0.170197,0.097633,0.043362,0.077426,0.111514,0.292666,0.020340,0.188615,...,0.083494,0.096437,0.154939,0.183300,0.113200,0.125523,0.140313,0.246185,0.304187,0.059267
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.145232,0.673895,0.282807,0.003281,0.122538,0.283951,0.306748,0.006117,0.133036,...,0.418981,0.129070,0.245059,0.571669,0.402439,0.088663,0.197183,0.113043,0.203488,0.008011
50%,0.500000,0.260041,0.787466,0.373626,0.010413,0.151714,0.335097,0.368098,0.009575,0.236110,...,0.458333,0.182946,0.328063,0.666293,0.456098,0.156977,0.267606,0.208696,0.325581,0.021876
75%,0.750000,0.355253,0.861936,0.424860,0.030267,0.189643,0.370370,0.496933,0.014564,0.379348,...,0.490741,0.233721,0.426877,0.762318,0.509756,0.226744,0.338028,0.408696,0.662791,0.048158
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [157]:
Placement_hot_coded = Placement_hot_coded.drop(['sl_no', 'status_Placed'], axis=1, errors="ignore")


CC_data_hot_coded = CC_data_hot_coded.drop(["index", "unitid", "chronname", "city", "site", "similar", "nicknames"], axis=1, errors="ignore")


In [158]:
# target variable for Placement

Placement_hot_coded["salary_target"] = (Placement_hot_coded["salary"] > Placement_hot_coded["salary"].median()).astype(int)


# target variable for CC_data

CC_data_hot_coded["grad_target"] = (CC_data_hot_coded["grad_150_value"] > CC_data_hot_coded["grad_150_value"].median()).astype(int)





In [159]:
# Prevalence
prev = (Placement_hot_coded["salary_target"].value_counts()[1] / len(Placement_hot_coded["salary_target"]))
print("prev", prev)

prev = (CC_data_hot_coded["grad_target"].value_counts()[1] / len(CC_data_hot_coded["grad_target"]))
print("prev", prev)



prev 0.3209302325581395
prev 0.4557661927330174


In [160]:
cc_dt = CC_data_hot_coded.dropna(subset=["grad_target"])

Train, Temp = train_test_split(cc_dt, train_size=0.55, stratify=cc_dt["grad_target"])

Tune, Test = train_test_split(Temp, test_size=0.50, stratify=Temp["grad_target"])




In [161]:
place_dt = Placement_hot_coded.dropna(subset=["salary_target"])

Train, Temp = train_test_split(place_dt, train_size=0.50, stratify=place_dt["salary_target"])

Tune, Test = train_test_split(Temp, test_size=0.50, stratify=Temp["salary_target"])


# Step three: What do your instincts tell you about the data. Can it address your problem, what areas/items are you worried about?

One main concern is that many salaries are missing from the dataset, which can substantially affect results and introduce selection bias. Another concern is the small sample size of observed salaries (about 150 rows), which reduces statistical power and increases the risk of overfitting. For the CC dataset, there are many missing values in some fields; although the dataset is large overall, these gaps may limit the variables you can reliably use.